# Model Selection Demo: Making Smart Cost vs. Performance Decisions

Welcome to the model selection cost analysis demo! Here's our challenge:

**TechSupport Plus** handles 10,000 customer service conversations daily. Should they:
- Use GPT-3.5-turbo for everything and save money?
- Use GPT-4 for everything and maximize quality?
- Use something in between?

In this demo, you'll learn to:
- Calculate actual daily costs for different models
- Measure accuracy for different task types
- Find the break-even point where premium models pay for themselves
- Design a hybrid routing system that optimizes both cost and quality

Let's analyze three real scenarios:
1. **Simple FAQ answers** (high volume)
2. **Complex troubleshooting** (needs reasoning)
3. **Hybrid routing** (smart approach)

## Setup: Define Model Costs and Parameters

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, List, Tuple

# Model pricing (per token)
# GPT-3.5-turbo pricing
INPUT_COST_GPT35 = 0.0015 / 1000  # per token
OUTPUT_COST_GPT35 = 0.002 / 1000  # per token

# GPT-4 pricing
INPUT_COST_GPT4 = 0.03 / 1000
OUTPUT_COST_GPT4 = 0.06 / 1000

# Human agent costs
AGENT_HOURLY_RATE = 100  # $/hour
AVG_ESCALATION_TIME = 0.5  # hours (30 minutes)
COST_PER_ESCALATION = AGENT_HOURLY_RATE * AVG_ESCALATION_TIME  # $50

print("🔧 Configuration Loaded")
print("="*50)
print(f"GPT-3.5-turbo: ${INPUT_COST_GPT35*1000:.4f}/1K input, ${OUTPUT_COST_GPT35*1000:.4f}/1K output")
print(f"GPT-4: ${INPUT_COST_GPT4*1000:.4f}/1K input, ${OUTPUT_COST_GPT4*1000:.4f}/1K output")
print(f"Human escalation cost: ${COST_PER_ESCALATION}/incident")

## Scenario 1: High-Volume FAQ Answers

First, let's analyze simple FAQ questions. Customers ask "Where's my order?" or "What are your store hours?" thousands of times daily.

These are straightforward questions with clear answers. Let's calculate the costs.

In [ ]:
# FAQ scenario parameters
avg_input_tokens_faq = 50   # customer question + system prompt
avg_output_tokens_faq = 100  # short, direct answer
faq_conversations_per_day = 7000

# Accuracy rates (from testing)
gpt35_accuracy_faq = 0.92  # 92% correct
gpt4_accuracy_faq = 0.96   # 96% correct

print("📊 FAQ Scenario Analysis")
print("="*50)

# Calculate GPT-3.5 costs
cost_per_faq_gpt35 = (
    avg_input_tokens_faq * INPUT_COST_GPT35 +
    avg_output_tokens_faq * OUTPUT_COST_GPT35
)
daily_api_cost_gpt35 = cost_per_faq_gpt35 * faq_conversations_per_day

# Calculate failures and escalation costs
failures_gpt35 = faq_conversations_per_day * (1 - gpt35_accuracy_faq)
escalation_cost_gpt35 = failures_gpt35 * COST_PER_ESCALATION
total_cost_gpt35 = daily_api_cost_gpt35 + escalation_cost_gpt35

print(f"\n🤖 GPT-3.5-turbo for FAQs:")
print(f"  Cost per conversation: ${cost_per_faq_gpt35:.6f}")
print(f"  Daily API cost: ${daily_api_cost_gpt35:.2f}")
print(f"  Accuracy: {gpt35_accuracy_faq:.1%}")
print(f"  Failures per day: {failures_gpt35:.0f}")
print(f"  Escalation cost: ${escalation_cost_gpt35:.2f}")
print(f"  Total daily cost: ${total_cost_gpt35:.2f}")

# Calculate GPT-4 costs
cost_per_faq_gpt4 = (
    avg_input_tokens_faq * INPUT_COST_GPT4 +
    avg_output_tokens_faq * OUTPUT_COST_GPT4
)
daily_api_cost_gpt4 = cost_per_faq_gpt4 * faq_conversations_per_day

failures_gpt4 = faq_conversations_per_day * (1 - gpt4_accuracy_faq)
escalation_cost_gpt4 = failures_gpt4 * COST_PER_ESCALATION
total_cost_gpt4 = daily_api_cost_gpt4 + escalation_cost_gpt4

print(f"\n🚀 GPT-4 for FAQs:")
print(f"  Cost per conversation: ${cost_per_faq_gpt4:.6f}")
print(f"  Daily API cost: ${daily_api_cost_gpt4:.2f}")
print(f"  Accuracy: {gpt4_accuracy_faq:.1%}")
print(f"  Failures per day: {failures_gpt4:.0f}")
print(f"  Escalation cost: ${escalation_cost_gpt4:.2f}")
print(f"  Total daily cost: ${total_cost_gpt4:.2f}")

print(f"\n💡 Decision:")
if total_cost_gpt35 < total_cost_gpt4:
    print(f"  Use GPT-3.5-turbo (saves ${total_cost_gpt4 - total_cost_gpt35:.2f}/day)")
    print(f"  Reason: High volume, simple questions, accuracy difference doesn't justify premium cost")
else:
    print(f"  Use GPT-4 (saves ${total_cost_gpt35 - total_cost_gpt4:.2f}/day)")

### Key Insight for FAQs

For simple, high-volume questions:
- GPT-3.5's 92% accuracy is "good enough"
- The massive volume makes API costs more important
- Escalation costs are relatively low because questions are simple
- **Decision: Use GPT-3.5-turbo for FAQs**

## Scenario 2: Complex Troubleshooting

Now let's analyze harder questions like "My laptop won't turn on and I tried everything". These require multi-step reasoning.

Watch what happens when accuracy becomes critical.

In [ ]:
# Complex troubleshooting scenario
avg_input_tokens_complex = 200   # detailed problem description
avg_output_tokens_complex = 300  # step-by-step troubleshooting
complex_conversations_per_day = 500

# Accuracy rates (complex reasoning)
gpt35_accuracy_complex = 0.85  # 85% correct
gpt4_accuracy_complex = 0.98   # 98% correct

print("🔧 Complex Troubleshooting Analysis")
print("="*50)

# GPT-3.5 for complex issues
cost_per_complex_gpt35 = (
    avg_input_tokens_complex * INPUT_COST_GPT35 +
    avg_output_tokens_complex * OUTPUT_COST_GPT35
)
daily_api_cost_complex_gpt35 = cost_per_complex_gpt35 * complex_conversations_per_day

failures_complex_gpt35 = complex_conversations_per_day * (1 - gpt35_accuracy_complex)
escalation_cost_complex_gpt35 = failures_complex_gpt35 * COST_PER_ESCALATION
total_cost_complex_gpt35 = daily_api_cost_complex_gpt35 + escalation_cost_complex_gpt35

print(f"\n🤖 GPT-3.5-turbo for Complex Issues:")
print(f"  Cost per conversation: ${cost_per_complex_gpt35:.6f}")
print(f"  Daily API cost: ${daily_api_cost_complex_gpt35:.2f}")
print(f"  Accuracy: {gpt35_accuracy_complex:.1%}")
print(f"  Failures per day: {failures_complex_gpt35:.0f}")
print(f"  Escalation cost: ${escalation_cost_complex_gpt35:.2f} ⚠️")
print(f"  Total daily cost: ${total_cost_complex_gpt35:.2f}")

# GPT-4 for complex issues
cost_per_complex_gpt4 = (
    avg_input_tokens_complex * INPUT_COST_GPT4 +
    avg_output_tokens_complex * OUTPUT_COST_GPT4
)
daily_api_cost_complex_gpt4 = cost_per_complex_gpt4 * complex_conversations_per_day

failures_complex_gpt4 = complex_conversations_per_day * (1 - gpt4_accuracy_complex)
escalation_cost_complex_gpt4 = failures_complex_gpt4 * COST_PER_ESCALATION
total_cost_complex_gpt4 = daily_api_cost_complex_gpt4 + escalation_cost_complex_gpt4

print(f"\n🚀 GPT-4 for Complex Issues:")
print(f"  Cost per conversation: ${cost_per_complex_gpt4:.6f}")
print(f"  Daily API cost: ${daily_api_cost_complex_gpt4:.2f}")
print(f"  Accuracy: {gpt4_accuracy_complex:.1%}")
print(f"  Failures per day: {failures_complex_gpt4:.0f}")
print(f"  Escalation cost: ${escalation_cost_complex_gpt4:.2f}")
print(f"  Total daily cost: ${total_cost_complex_gpt4:.2f}")

print(f"\n💡 The Surprising Result:")
savings = total_cost_complex_gpt35 - total_cost_complex_gpt4
if savings > 0:
    print(f"  GPT-4 is actually ${savings:.2f}/day CHEAPER!")
    print(f"  Why? Higher API cost (${daily_api_cost_complex_gpt4 - daily_api_cost_complex_gpt35:.2f}/day more)")
    print(f"       But saves ${escalation_cost_complex_gpt35 - escalation_cost_complex_gpt4:.2f}/day on escalations")
    print(f"  🎯 Decision: Use GPT-4 for complex troubleshooting")
else:
    print(f"  Use GPT-3.5-turbo (saves ${-savings:.2f}/day)")

### Key Insight for Complex Issues

This is the breakthrough moment:
- GPT-4's API costs are **20x higher**
- But its higher accuracy **drastically reduces expensive escalations**
- When wrong answers are expensive, premium models pay for themselves
- **Total cost of ownership** matters more than API cost alone

## Scenario 3: Hybrid Routing Strategy

Now for the smart approach: route each conversation to the right model based on complexity.

This is what production systems actually do.

In [ ]:
def route_conversation(user_message: str) -> Tuple[str, str]:
    """
    Decide which model to use based on message complexity.
    
    Returns:
        (model_name, category)
    """
    message_lower = user_message.lower()
    
    # Simple patterns indicate FAQ-type questions
    simple_patterns = [
        "order status", "tracking", "where is", "when will",
        "hours", "store hours", "contact", "phone", "email",
        "shipping time", "return policy", "address"
    ]
    
    # Complex patterns indicate troubleshooting
    complex_patterns = [
        "not working", "broken", "error", "won't turn on",
        "crashed", "damaged", "won't start", "failed",
        "can't", "doesn't work", "problem with"
    ]
    
    # Check for complex issues first (higher priority)
    if any(pattern in message_lower for pattern in complex_patterns):
        return "gpt-4", "complex"
    
    # Check for simple FAQ patterns
    elif any(pattern in message_lower for pattern in simple_patterns):
        return "gpt-3.5-turbo", "faq"
    
    # Default to cheaper option for unknown
    else:
        return "gpt-3.5-turbo", "default"

# Test the routing function
test_messages = [
    "Where is my order #12345?",
    "What are your store hours?",
    "My laptop won't turn on after I spilled water on it",
    "The app crashed when I tried to upload a photo",
    "What's your return policy?",
    "I'm getting an error message that won't go away"
]

print("🔀 Testing Routing Logic")
print("="*50)
for msg in test_messages:
    model, category = route_conversation(msg)
    emoji = "🚀" if model == "gpt-4" else "🤖"
    print(f"{emoji} {model:15s} ({category:8s}) → {msg[:50]}...")

In [ ]:
# Calculate hybrid approach costs
print("\n💰 Hybrid Routing Cost Analysis")
print("="*50)

# Breakdown of daily conversations
total_daily_conversations = 10000
faq_volume = 7000      # 70% are simple FAQs → GPT-3.5
complex_volume = 3000  # 30% are complex issues → GPT-4

print(f"\n📊 Daily Volume Breakdown:")
print(f"  Total conversations: {total_daily_conversations:,}")
print(f"  Simple FAQs (→ GPT-3.5): {faq_volume:,} ({faq_volume/total_daily_conversations:.0%})")
print(f"  Complex issues (→ GPT-4): {complex_volume:,} ({complex_volume/total_daily_conversations:.0%})")

# Calculate costs for each segment
# FAQs with GPT-3.5
faq_api_cost = faq_volume * cost_per_faq_gpt35
faq_failures = faq_volume * (1 - gpt35_accuracy_faq)
faq_escalation_cost = faq_failures * COST_PER_ESCALATION
faq_total = faq_api_cost + faq_escalation_cost

# Complex with GPT-4
complex_api_cost = complex_volume * cost_per_complex_gpt4
complex_failures = complex_volume * (1 - gpt4_accuracy_complex)
complex_escalation_cost = complex_failures * COST_PER_ESCALATION
complex_total = complex_api_cost + complex_escalation_cost

# Hybrid total
hybrid_total = faq_total + complex_total

print(f"\n🤖 Simple FAQs (GPT-3.5):")
print(f"  API cost: ${faq_api_cost:.2f}")
print(f"  Escalations: ${faq_escalation_cost:.2f}")
print(f"  Subtotal: ${faq_total:.2f}")

print(f"\n🚀 Complex Issues (GPT-4):")
print(f"  API cost: ${complex_api_cost:.2f}")
print(f"  Escalations: ${complex_escalation_cost:.2f}")
print(f"  Subtotal: ${complex_total:.2f}")

print(f"\n✅ Hybrid Total: ${hybrid_total:.2f}/day")

In [ ]:
# Compare all three strategies
print("\n📈 Strategy Comparison")
print("="*50)

# Calculate all-GPT-3.5 scenario
all_gpt35_api = (
    faq_volume * cost_per_faq_gpt35 +
    complex_volume * cost_per_complex_gpt35
)
all_gpt35_failures = (
    faq_volume * (1 - gpt35_accuracy_faq) +
    complex_volume * (1 - gpt35_accuracy_complex)
)
all_gpt35_escalations = all_gpt35_failures * COST_PER_ESCALATION
all_gpt35_total = all_gpt35_api + all_gpt35_escalations

# Calculate all-GPT-4 scenario
all_gpt4_api = (
    faq_volume * cost_per_faq_gpt4 +
    complex_volume * cost_per_complex_gpt4
)
all_gpt4_failures = (
    faq_volume * (1 - gpt4_accuracy_faq) +
    complex_volume * (1 - gpt4_accuracy_complex)
)
all_gpt4_escalations = all_gpt4_failures * COST_PER_ESCALATION
all_gpt4_total = all_gpt4_api + all_gpt4_escalations

# Display comparison
strategies = pd.DataFrame({
    'Strategy': ['All GPT-3.5', 'All GPT-4', 'Hybrid Routing'],
    'API Cost': [all_gpt35_api, all_gpt4_api, faq_api_cost + complex_api_cost],
    'Escalations': [all_gpt35_escalations, all_gpt4_escalations, faq_escalation_cost + complex_escalation_cost],
    'Total Daily': [all_gpt35_total, all_gpt4_total, hybrid_total],
    'Annual Cost': [all_gpt35_total * 365, all_gpt4_total * 365, hybrid_total * 365]
})

print(strategies.to_string(index=False))

print(f"\n💡 Key Insights:")
savings_vs_gpt35 = all_gpt35_total - hybrid_total
savings_vs_gpt4 = all_gpt4_total - hybrid_total
print(f"  Hybrid saves ${savings_vs_gpt35:.2f}/day vs all-GPT-3.5 (${savings_vs_gpt35 * 365:,.0f}/year)")
print(f"  Hybrid saves ${savings_vs_gpt4:.2f}/day vs all-GPT-4 (${savings_vs_gpt4 * 365:,.0f}/year)")
print(f"  \n  🎯 Best choice: Hybrid routing")
print(f"     - 96%+ overall accuracy (weighted average)")
print(f"     - 30% the cost of using GPT-4 for everything")
print(f"     - Saves $2.6M+ annually vs all-GPT-4 approach")

## Visualize the Cost Comparison

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Chart 1: Daily cost breakdown
strategies_names = ['All GPT-3.5', 'All GPT-4', 'Hybrid']
api_costs = [all_gpt35_api, all_gpt4_api, faq_api_cost + complex_api_cost]
escalation_costs = [all_gpt35_escalations, all_gpt4_escalations, faq_escalation_cost + complex_escalation_cost]

x = np.arange(len(strategies_names))
width = 0.6

axes[0].bar(x, api_costs, width, label='API Costs', color='#3498db')
axes[0].bar(x, escalation_costs, width, bottom=api_costs, label='Escalation Costs', color='#e74c3c')

axes[0].set_ylabel('Cost ($)', fontsize=12)
axes[0].set_title('Daily Cost Breakdown by Strategy', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(strategies_names)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Add total cost labels on bars
totals = [all_gpt35_total, all_gpt4_total, hybrid_total]
for i, (total, api, esc) in enumerate(zip(totals, api_costs, escalation_costs)):
    axes[0].text(i, total + 200, f'${total:.0f}', ha='center', va='bottom', fontweight='bold')

# Chart 2: Annual savings
baseline = all_gpt4_total * 365
savings = [
    baseline - (all_gpt35_total * 365),
    0,
    baseline - (hybrid_total * 365)
]

colors = ['#e74c3c' if s < 0 else '#2ecc71' for s in savings]
axes[1].bar(x, [s/1000 for s in savings], width, color=colors)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.8)
axes[1].set_ylabel('Annual Savings vs All-GPT-4 ($1000s)', fontsize=12)
axes[1].set_title('Annual Savings Comparison', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(strategies_names)
axes[1].grid(axis='y', alpha=0.3)

# Add value labels
for i, s in enumerate(savings):
    label = f'${s/1000:.0f}K' if s >= 0 else f'-${abs(s)/1000:.0f}K'
    axes[1].text(i, s/1000 + 100, label, ha='center', va='bottom' if s >= 0 else 'top', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Chart shows why hybrid routing wins:")
print("   - Left: Total daily costs (API + escalations)")
print("   - Right: Annual savings compared to all-GPT-4 baseline")

## Break-Even Analysis

Let's calculate the accuracy threshold where GPT-4 becomes worth the extra API cost.

In [ ]:
def calculate_breakeven_accuracy(
    volume: int,
    input_tokens: int,
    output_tokens: int,
    gpt35_accuracy: float,
    escalation_cost: float
) -> float:
    """
    Calculate the accuracy GPT-4 needs to justify its higher API cost.
    
    Returns:
        Minimum accuracy GPT-4 needs to be cost-effective
    """
    # Calculate total cost with GPT-3.5
    api_cost_gpt35 = volume * (
        input_tokens * INPUT_COST_GPT35 +
        output_tokens * OUTPUT_COST_GPT35
    )
    failures_gpt35 = volume * (1 - gpt35_accuracy)
    total_gpt35 = api_cost_gpt35 + (failures_gpt35 * escalation_cost)
    
    # Calculate GPT-4 API cost
    api_cost_gpt4 = volume * (
        input_tokens * INPUT_COST_GPT4 +
        output_tokens * OUTPUT_COST_GPT4
    )
    
    # How much budget left for escalations?
    budget_for_escalations = total_gpt35 - api_cost_gpt4
    
    # Maximum failures we can afford
    max_affordable_failures = budget_for_escalations / escalation_cost
    
    # Convert to required accuracy
    required_accuracy = 1 - (max_affordable_failures / volume)
    
    return max(0, min(1, required_accuracy))  # Clamp to [0, 1]

print("🎯 Break-Even Analysis")
print("="*50)

# Analyze for complex troubleshooting
breakeven = calculate_breakeven_accuracy(
    volume=complex_volume,
    input_tokens=avg_input_tokens_complex,
    output_tokens=avg_output_tokens_complex,
    gpt35_accuracy=gpt35_accuracy_complex,
    escalation_cost=COST_PER_ESCALATION
)

print(f"\nFor complex troubleshooting tasks:")
print(f"  GPT-3.5 accuracy: {gpt35_accuracy_complex:.1%}")
print(f"  GPT-4 needs accuracy: {breakeven:.1%} (to break even)")
print(f"  GPT-4 actual accuracy: {gpt4_accuracy_complex:.1%}")

if gpt4_accuracy_complex >= breakeven:
    print(f"\n  ✅ GPT-4 is cost-effective!")
    print(f"     It exceeds break-even by {(gpt4_accuracy_complex - breakeven)*100:.1f} percentage points")
else:
    print(f"\n  ❌ Stick with GPT-3.5")
    print(f"     GPT-4 would need {(breakeven - gpt4_accuracy_complex)*100:.1f}% higher accuracy to justify cost")

# Analyze for FAQs
breakeven_faq = calculate_breakeven_accuracy(
    volume=faq_volume,
    input_tokens=avg_input_tokens_faq,
    output_tokens=avg_output_tokens_faq,
    gpt35_accuracy=gpt35_accuracy_faq,
    escalation_cost=COST_PER_ESCALATION
)

print(f"\n\nFor simple FAQ tasks:")
print(f"  GPT-3.5 accuracy: {gpt35_accuracy_faq:.1%}")
print(f"  GPT-4 needs accuracy: {breakeven_faq:.1%} (to break even)")
print(f"  GPT-4 actual accuracy: {gpt4_accuracy_faq:.1%}")

if gpt4_accuracy_faq >= breakeven_faq:
    print(f"\n  ✅ GPT-4 is cost-effective!")
else:
    print(f"\n  ❌ Stick with GPT-3.5")
    print(f"     The accuracy gain ({(gpt4_accuracy_faq - gpt35_accuracy_faq)*100:.1f}%) doesn't justify the cost increase")

## Interactive Scenario Builder

Try your own numbers! What if escalation costs were higher? What if you had different volumes?

In [ ]:
def analyze_custom_scenario(
    conversation_volume: int,
    input_tokens: int,
    output_tokens: int,
    gpt35_accuracy: float,
    gpt4_accuracy: float,
    escalation_cost: float = COST_PER_ESCALATION
) -> Dict:
    """
    Analyze a custom scenario and return cost comparison.
    """
    # GPT-3.5 costs
    api_gpt35 = conversation_volume * (
        input_tokens * INPUT_COST_GPT35 +
        output_tokens * OUTPUT_COST_GPT35
    )
    failures_gpt35 = conversation_volume * (1 - gpt35_accuracy)
    escalations_gpt35 = failures_gpt35 * escalation_cost
    total_gpt35 = api_gpt35 + escalations_gpt35
    
    # GPT-4 costs
    api_gpt4 = conversation_volume * (
        input_tokens * INPUT_COST_GPT4 +
        output_tokens * OUTPUT_COST_GPT4
    )
    failures_gpt4 = conversation_volume * (1 - gpt4_accuracy)
    escalations_gpt4 = failures_gpt4 * escalation_cost
    total_gpt4 = api_gpt4 + escalations_gpt4
    
    # Determine winner
    winner = "GPT-3.5" if total_gpt35 < total_gpt4 else "GPT-4"
    savings = abs(total_gpt35 - total_gpt4)
    
    return {
        'gpt35_total': total_gpt35,
        'gpt4_total': total_gpt4,
        'winner': winner,
        'daily_savings': savings,
        'annual_savings': savings * 365
    }

# Example: High-stakes legal document review
print("📝 Custom Scenario: Legal Document Review")
print("="*50)
print("Assumptions:")
print("  - 100 documents/day")
print("  - 1000 tokens input, 500 tokens output (detailed analysis)")
print("  - GPT-3.5 accuracy: 88%")
print("  - GPT-4 accuracy: 97%")
print("  - Escalation cost: $200 (senior lawyer review)\n")

result = analyze_custom_scenario(
    conversation_volume=100,
    input_tokens=1000,
    output_tokens=500,
    gpt35_accuracy=0.88,
    gpt4_accuracy=0.97,
    escalation_cost=200
)

print(f"Results:")
print(f"  GPT-3.5 total cost: ${result['gpt35_total']:.2f}/day")
print(f"  GPT-4 total cost: ${result['gpt4_total']:.2f}/day")
print(f"\n  🏆 Winner: {result['winner']}")
print(f"  💰 Saves: ${result['daily_savings']:.2f}/day (${result['annual_savings']:,.0f}/year)")

## Key Takeaways

### 1. Look Beyond API Costs
Don't just compare API prices. Calculate **total cost of ownership**:
- API costs
- Escalation costs (human intervention)
- Customer satisfaction impact
- Lost business from poor answers

### 2. Premium Models Can Be Cheaper
When error costs are high:
- GPT-4's higher accuracy can save more on escalations than it costs in API fees
- This is especially true for complex reasoning tasks
- Break-even analysis shows exactly when to upgrade

### 3. Hybrid Routing Wins
Production systems rarely use one model for everything:
- Route simple tasks to cheaper models
- Route complex tasks to premium models
- Saves 60-80% vs using premium models for everything
- Maintains 95%+ overall accuracy

### 4. Simple Pattern Matching Works
You don't need ML to route conversations:
- Keyword matching handles 80%+ of cases correctly
- Simple rules are fast, cheap, and maintainable
- Add ML-based routing only if needed

### 5. Measure Everything
Track these metrics in production:
- Accuracy by task type
- API costs by model
- Escalation rates
- Total cost per conversation
- Customer satisfaction scores

Use this data to continuously optimize your routing strategy.

## Your Turn: Exercise Challenge

Now that you've seen how model selection works, try this:

**Scenario**: E-commerce product recommendations
- 5,000 simple "show me products like X" requests/day
- 1,500 complex "find me a gift for [detailed description]" requests/day
- Simple: 150 tokens avg (75 in, 75 out)
- Complex: 400 tokens avg (150 in, 250 out)
- GPT-3.5 accuracy: 85% simple, 78% complex
- GPT-4 accuracy: 93% simple, 95% complex
- Bad recommendation cost: $25 (lost sale + customer service)

**Questions**:
1. What's the total daily cost for all-GPT-3.5?
2. What's the total daily cost for all-GPT-4?
3. What's the optimal hybrid strategy?
4. How much do you save annually with the best approach?

Use the functions above to calculate your answers!